# Rock-Paper-Scissors
Training a CNN on rock-paper-scissors hand images to build a simple game application

In [ ]:
# import modules -----------------------------------------------------------------------------------
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
from keras import callbacks
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import InputLayer, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# define paths -------------------------------------------------------------------------------------
train_folder = 'dataset/train'
test_folder = 'dataset/test'
validation_folder = 'dataset/validation'

# Helper methods -----------------------------------------------------------------------------------
def load_image(img_path: str) -> np.ndarray:
    '''Load an image from a file path and return it as a numpy array.'''
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_rgb

def display_image(image: np.ndarray) -> None:
    '''Display an image in a matplotlib window.'''
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def save_rps_model(model: Sequential, name: str) -> None:
    '''Save a model to a file with a timestamped filename.'''
    timestamp = datetime.now().strftime('%Y%m%d_%H%M')
    filename = f'models/{name}_{timestamp}.h5'
    model.save(filename)

def load_rps_model(model_path: str = None):
    '''Load a model from a file path. If no path is provided, the latest model is loaded.'''

    if not os.path.isfile(str(model_path)):
        # load latest version
        model_path = os.path.join('models', sorted([m for m in os.listdir('models') if m.lower().endswith('.h5')])[-1])

    model = load_model(model_path)
    input_shape = model.layers[0].input_shape

    print(f'Loaded model: {model_path} with input shape: {input_shape}')
    return model

def get_random_picture(set: str = None) -> str:
    '''Return a random picture path from the train, test or validation set.'''

    if set == 'train':
        folder = train_folder
    elif set == 'test':
        folder = test_folder
    elif set == 'validation':
        folder = validation_folder
    else:
        folder = random.choice([train_folder, test_folder, validation_folder])
    
    subfolder = random.choice(os.listdir(folder))
    subfolder_path = os.path.join(folder, subfolder)
    test_image = random.choice(os.listdir(subfolder_path))
    image_path = os.path.normpath(os.path.join(subfolder_path, test_image))

    return image_path

## Check dataset content

In [ ]:
# open test picture
image_path = get_random_picture()
print('Image:', image_path)
img = load_image(image_path)
display_image(img)

## Preparing data for the model

In [ ]:
image_shape = (150, 150, 3)
batch_size = 32

image_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,  # True if you plan to show your hand upside down
    fill_mode='nearest'
)

train_image_gen = image_gen.flow_from_directory(
    train_folder,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True  # enable shuffling to make the model more robust
)

test_image_gen = image_gen.flow_from_directory(
    test_folder,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # disable shuffling to keep data in same order as labels
)

validation_generator = image_gen.flow_from_directory(
        validation_folder,
        target_size=image_shape[:2],
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False)  # disable shuffling to keep data in same order as labels

In [ ]:
# test generator on random image
random_img = image_gen.random_transform(img)
display_image(random_img)

## Create Model

In [ ]:
# create sequential modle
model = Sequential()

model.add(InputLayer(input_shape=image_shape))

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))

# Add 50% dropout to help reduce overfitting
model.add(Dropout(0.5))

# Output layer
model.add(Dense(units=3, activation='softmax'))  # 3 classes

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

## Moving the model to GPU
WIP, I currently cannot see my GPU listed here

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Training Model

In [ ]:
# indices
class_indices = train_image_gen.class_indices
class_labels = {v:k for k, v in class_indices.items()}
print(class_indices)
print(class_labels)

In [ ]:
# Initializing early stopping callback to monitor the validation loss and prevent overfitting
earlystopping = callbacks.EarlyStopping(monitor="accuracy",
                                        mode="max",
                                        patience=10,
                                        restore_best_weights=True)

In [ ]:
# Train the model
results = model.fit(
    train_image_gen,
    epochs=100,
    steps_per_epoch=train_image_gen.samples/train_image_gen.batch_size,
    validation_data=test_image_gen,
    validation_steps=test_image_gen.samples/test_image_gen.batch_size,
    verbose=2,
    callbacks=[earlystopping]
    )

## Visualize Accuracy

In [ ]:
plt.plot(results.history['accuracy'])

## Save model

In [ ]:
save_rps_model(model, 'rps_v07_47[epoch]_0.9585[acc]_0.1195[loss]')

## Confusion Matrix and Classification Report

In [ ]:
model = load_rps_model('models/rps_v06_56[epoch]_0.9641[acc]_0.1089[loss].h5')

validation_steps = len(validation_generator)
validation_generator.reset() # Reset the generator to be sure of avoiding shuffling

predictions = model.predict_generator(validation_generator, steps=validation_steps, verbose=1)
y_pred_classes = np.argmax(predictions, axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(validation_generator.classes, y_pred_classes))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(validation_generator.classes, y_pred_classes))

## Image prediction Test

In [ ]:
model = load_rps_model()
input_model_size = model.layers[0].input_shape[1:3]

class_indices = {'paper': 0, 'rock': 1, 'scissors': 2}
class_labels = {v:k for k, v in class_indices.items()}

image_path = get_random_picture()
print('Image:', image_path)
img = image.load_img(image_path, target_size=input_model_size)
display_image(img)

img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255

prediction_prob = model.predict(img, verbose=0)[0]
predicted_class_index = np.argmax(prediction_prob)
predicted_class_label = class_labels[predicted_class_index]
predicted_class_prob = prediction_prob[predicted_class_index]

print('prediction_prob: ', [f'{k}: {prediction_prob[v]}' for k, v in class_indices.items()])
print('predicted_class_index:', predicted_class_index)
print('predicted_class_label:', predicted_class_label)
print('predicted_class_prob:', predicted_class_prob)

## Classify Validation Pictures
Run all validation pictures through the model and classify them. Display the failed ones.

In [ ]:
model = load_rps_model()
input_model_size = model.layers[0].input_shape[1:3]

success = 0
failure = 0
total = 0
failed_images = []

# load model
class_indices = {'paper': 0, 'rock': 1, 'scissors': 2}
class_labels = {v:k for k, v in class_indices.items()}

test_images = None  # limit number of images to test if needed, None to go through all of them
idx = 0

for subfolder in ['rock', 'paper', 'scissors']:

    current_folder = os.path.join(validation_folder, subfolder)

    for img_name in os.listdir(current_folder):

        if test_images is not None and idx > test_images - 1:
            break
        idx += 1

        img_file = os.path.normpath(os.path.join(current_folder, img_name))

        roi_copy = image.load_img(img_file, target_size=input_model_size)
        img_array = image.img_to_array(roi_copy)

        # add batch dimension
        img_normalized = img_array/255
        img_expanded = np.expand_dims(img_normalized, axis=0)

        # image prediction probability
        prediction_prob = model.predict(img_expanded, verbose=0)

        # Get predicted class index
        predicted_class_index = np.argmax(prediction_prob)

        # Map predicted class index to class label
        predicted_class_label = class_labels[predicted_class_index]

        # Print predicted class label
        if subfolder == predicted_class_label:
            success += 1
            result = 'success'
        else:
            failure += 1
            result = 'failure'
            failed_images.append((img_normalized, predicted_class_label, img_file))

        total += 1
        to_print = f'Images Classified: {total} | Success: {success} | Failure: {failure} | Accuracy: {success/total*100:.2f}%'
        print(f'{to_print}', end='\r', flush=True)

# print failed images with prediction and path
if failed_images:
    print(f"\n{50 * '-'}\nFAILED IMAGES:")
    for roi_copy, label, model_path in failed_images:
        print(model_path)
        img_size = 200
        plt.figure(figsize=(img_size/100, img_size/100))
        plt.imshow(roi_copy, cmap='gray')
        plt.title(label, loc='left')
        plt.axis('off')
        plt.show()

## Test on camera
Define a ROI from webcam frames and predict the class.

In [ ]:
model = load_rps_model()
input_model_size = model.layers[0].input_shape[1:3]

class_labels = {0: 'paper', 1: 'rock', 2: 'scissors'}

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cv2.namedWindow('Rock Paper Scissors')

# define ROI position and size
x, y, w, h = 895, 78, 300, 300

while True:

    try:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        roi = frame[y:y+h, x:x+w]

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

        # convert roi to RGB, resize and normalize
        roi_resized = cv2.resize(roi, input_model_size, interpolation=cv2.INTER_AREA)
        roi_rgb = cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB)
        roi_normalized = roi_rgb / 255.0
        roi_expanded = np.expand_dims(roi_normalized, axis=0)

        # Classify the ROI image
        prediction_prob = model.predict(roi_expanded, verbose=0)[0]
        predicted_class_index = np.argmax(prediction_prob)
        predicted_class_label = class_labels[predicted_class_index]
        predicted_class_prob = prediction_prob[predicted_class_index]

        # display captured roi top left of the frame for debugging
        frame[0:input_model_size[0], 0:input_model_size[1]] = roi_resized

        # Display prediction
        if predicted_class_prob < 0.9:
            label = 'Undefined'
        else:
            label = f'{predicted_class_label} ({predicted_class_prob:.2f})'

        p_paper, p_rock, p_scissors = (f'{p:.2f}' for p in prediction_prob)
        print(f'Prediction: Rock {p_rock} - Paper {p_paper} - Scissors {p_scissors}', end='\r')

        # Draw label
        position = x, y + h + 20
        color = 0, 0, 255
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

        # Display frame
        cv2.imshow('Rock Paper Scissors', frame)

    except Exception as e:
        print(e)
        break

cap.release()
cv2.destroyAllWindows()